In [1]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movies =pd.read_csv('./tmdb_5000_movies.csv')
movies_df = movies[['id','title', 'genres', 'keywords']]

In [3]:
# keywords 테스트
#print(type(movies_df['keywords'][1]))

In [4]:
movies_df['genres'] = movies_df['genres'].apply(lambda x: json.loads(x))
#문자열 -> dict
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: json.loads(x))

In [5]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])
# 문자열 리스트 ? oo
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])

In [6]:
# 단어 벡터화 객체 생성
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))

In [7]:
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
movies_df['keywords_literal'] = movies_df['keywords'].apply(lambda x : (' ').join(x))

genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
keyword_mat = count_vect.fit_transform(movies_df['keywords_literal'])

In [9]:
movies_df['genres_literal'][:10]

0    Action Adventure Fantasy Science Fiction
1                    Adventure Fantasy Action
2                      Action Adventure Crime
3                 Action Crime Drama Thriller
4            Action Adventure Science Fiction
5                    Fantasy Action Adventure
6                            Animation Family
7            Action Adventure Science Fiction
8                    Adventure Fantasy Family
9                    Action Adventure Fantasy
Name: genres_literal, dtype: object

In [8]:
print(genre_mat)
print(type(genre_mat))

  (0, 0)	1
  (0, 16)	1
  (0, 124)	1
  (0, 232)	1
  (0, 138)	1
  (0, 1)	1
  (0, 24)	1
  (0, 135)	1
  (0, 233)	1
  (1, 0)	1
  (1, 16)	1
  (1, 124)	1
  (1, 24)	1
  (1, 125)	1
  (2, 0)	1
  (2, 16)	1
  (2, 1)	1
  (2, 64)	1
  (2, 20)	1
  (3, 0)	1
  (3, 64)	1
  (3, 90)	1
  (3, 234)	1
  (3, 4)	1
  (3, 68)	1
  :	:
  (4796, 90)	1
  (4796, 234)	1
  (4796, 106)	1
  (4796, 144)	1
  (4797, 234)	1
  (4797, 153)	1
  (4797, 157)	1
  (4798, 0)	1
  (4798, 64)	1
  (4798, 234)	1
  (4798, 4)	1
  (4798, 78)	1
  (4799, 44)	1
  (4799, 214)	1
  (4799, 58)	1
  (4800, 90)	1
  (4800, 44)	1
  (4800, 214)	1
  (4800, 104)	1
  (4800, 50)	1
  (4800, 250)	1
  (4800, 182)	1
  (4800, 251)	1
  (4800, 229)	1
  (4802, 80)	1
<class 'scipy.sparse.csr.csr_matrix'>


In [8]:
genre_sim = cosine_similarity(genre_mat, genre_mat)
# 장르 코사인 유사도
keyword_sim = cosine_similarity(keyword_mat, keyword_mat)


#여기서 테스트하면서 진행

In [10]:
#print(type(genre_sim))
print(keyword_sim)

[[1.         0.01628008 0.         ... 0.02614435 0.         0.        ]
 [0.01628008 1.         0.         ... 0.03277368 0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.02614435 0.03277368 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [11]:
# 장르 + 키워드 : 코사인 유사도
genre_keyword_sim = genre_sim + keyword_sim
print(genre_keyword_sim)
#genre_sorted_ind = genre_sim.argsort()[:, ::-1]

[[2.         0.61256488 0.4472136  ... 0.02614435 0.         0.        ]
 [0.61256488 2.         0.4        ... 0.03277368 0.         0.        ]
 [0.4472136  0.4        2.         ... 0.         0.         0.        ]
 ...
 [0.02614435 0.03277368 0.         ... 2.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         2.        ]]


In [12]:
# genre_keyword_sim
genre_keyword_sim_sorted_ind = genre_keyword_sim.argsort()[:, ::-1]
# 값 내림차 정렬, 인덱스 반환

In [13]:
# 사례로 테스트
title_movie = movies_df[movies_df['title'] == 'The Godfather']
title_index = title_movie.index.values #인덱스

# 상위 10개 index 가져오기
genre_keyword_similar_indexes = genre_keyword_sim_sorted_ind[title_index, :10]


# 1차원으로 변환
genre_keyword_similar_indexes = genre_keyword_similar_indexes.reshape(-1)

# index로 dataframe 가져오기
genre_keyword_similar_movies = movies_df.iloc[genre_keyword_similar_indexes]

In [16]:
print(genre_keyword_similar_movies[['title','genres', 'keywords']])

                                               title          genres  \
3337                                   The Godfather  [Drama, Crime]   
1946  The Bad Lieutenant: Port of Call - New Orleans  [Drama, Crime]   
1847                                      GoodFellas  [Drama, Crime]   
2731                          The Godfather: Part II  [Drama, Crime]   
4065                                      Mi America  [Drama, Crime]   
281                                American Gangster  [Drama, Crime]   
4217                                            Kids  [Drama, Crime]   
2640         Things to Do in Denver When You're Dead  [Drama, Crime]   
1243                                    Mean Streets  [Drama, Crime]   
2582                      The Place Beyond the Pines  [Drama, Crime]   

                                               keywords  
3337  [italy, love at first sight, loss of father, p...  
1946  [police brutality, organized crime, policeman,...  
1847  [prison, based on novel, fl